In [15]:
#####utils
##########

import numpy as np
import scipy.sparse as sp
import torch


def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot




def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


In [36]:
import numpy as np
import scipy.sparse as sp
import random
def load_data(path="/Users/stephenye/Desktop/2019summer/pygcn-master/data/MUTAG/", dataset="MUTAG"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    N = 188 ## num of graphs
    y = np.genfromtxt("{}{}_graph_labels.txt".format(path, dataset),dtype=np.dtype(int))
    idx_features = np.genfromtxt("{}{}_node_labels.txt".format(path, dataset),dtype=np.dtype(float))
    idx_indicator = np.genfromtxt("{}{}_graph_indicator.txt".format(path, dataset),dtype=np.dtype(int))
    Adj_M = np.genfromtxt("{}{}_A.txt".format(path, dataset),dtype=np.dtype(float),delimiter=",")
    
    for i in range(len(y)):
        if(y[i]==-1):
            y[i]= 0
    
    n = len(idx_features) ##num of nodes
    #create graph dictionary
    graph_node_dict = {}
    g_idx =0
    for i in range(len(idx_indicator)):
        if(g_idx != idx_indicator[i]):
            graph_node_dict[g_idx+1] = i
            g_idx +=1
    graph_node_dict[189] = len(idx_indicator)
    
    #build graphs
    A = np.zeros((n,n))
    for i in range(len(Adj_M)):
        A[int(Adj_M[i][0]-1)][int(Adj_M[i][1]-1)] = 1
        

    A = A + np.matmul(A.T,(A.T > A)) - np.matmul(A,(A.T > A))
    full = np.full((3371, 3371), 1)
    A = normalize(A + np.diag(np.diag(full)))
    
    idx_train = torch.tensor(range(110,188))
    idx_test = torch.tensor(range(30,110))
    idx_val = torch.tensor(range(30))
    
    A = torch.FloatTensor(A)
    y = torch.tensor(y)
    idx_features = encode_onehot(idx_features)
    idx_features = torch.FloatTensor(idx_features)
    return A,graph_node_dict,idx_features,y,idx_train,idx_test,idx_val


In [37]:
#####layers
###########


import math

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        #self.weight = self.weight.long()
        support = torch.matmul(input, self.weight)
        output = torch.matmul(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


In [38]:
#####GCN
########
import torch.nn as nn
import torch.nn.functional as F
##from pygcn.layers import GraphConvolution


class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj,diction):
        
        x = F.relu(self.gc1(x, adj))
        
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        x = F.log_softmax(x, dim=1)
        
        new_x = torch.tensor(np.zeros((188,2)))
        
        for i in range(188):  ##this may cause bug
            start = int(diction[i+1])
            end = int(diction[i+2])
            new_x[i] = torch.mean(x[start:end],0)
        output = new_x
        
        return new_x


In [39]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim


n_seed = 42
n_epochs = 200
_lr = 0.01
_weight_decay = 5e-4
n_hidden = 16
_dropout = 0.5
# Training settings
#parser = argparse.ArgumentParser()
#parser.add_argument('--no-cuda', action='store_true', default=False,
                    #help='Disables CUDA training.')
#parser.add_argument('--fastmode', action='store_true', default=False,
                    #help='Validate during training pass.')
#parser.add_argument('--seed', type=int, default=42, help='Random seed.')
#parser.add_argument('--epochs', type=int, default=200,
                    #help='Number of epochs to train.')
#parser.add_argument('--lr', type=float, default=0.01,
                    #help='Initial learning rate.')
#parser.add_argument('--weight_decay', type=float, default=5e-4,
                    #help='Weight decay (L2 loss on parameters).')
#parser.add_argument('--hidden', type=int, default=16,
                    #help='Number of hidden units.')
#parser.add_argument('--dropout', type=float, default=0.5,
                    #help='Dropout rate (1 - keep probability).')

#args = parser.parse_args()
#args.cuda = not args.no_cuda and torch.cuda.is_available()

np.random.seed(n_seed)
torch.manual_seed(n_seed)


# Load data
adj,g_dict,features,labels,idx_train, idx_test,idx_val = load_data()
torch.mean(features[3:5],0)
features[3:5]
adj

Loading MUTAG dataset...


tensor([[0.3333, 0.3333, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3333, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.2500, 0.2500, 0.2500],
        [0.0000, 0.0000, 0.0000,  ..., 0.5000, 0.5000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.5000, 0.0000, 0.5000]])

In [40]:
# Model and optimizer
model = GCN(nfeat=7,
            nhid=n_hidden,
            nclass=2,
            dropout=_dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=_lr, weight_decay=_weight_decay)

#if args.cuda:
#    model.cuda()
#    features = features.cuda()
#    adj = adj.cuda()
#   labels = labels.cuda()
#    idx_train = idx_train.cuda()
#    idx_val = idx_val.cuda()
#    idx_test = idx_test.cuda()


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj,g_dict)
    print(labels[idx_train])
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

#    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
#        model.eval()
#        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj,g_dict)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


# Train model
t_total = time.time()
for epoch in range(n_epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0001 loss_train: 0.8462 acc_train: 0.4359 loss_val: 0.9727 acc_val: 0.2667 time: 0.0820s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0002 loss_train: 0.8203 acc_train: 0.4359 loss_val: 0.9445 acc_val: 0.2667 time: 0.0625s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 

tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0024 loss_train: 0.6786 acc_train: 0.5641 loss_val: 0.6264 acc_val: 0.7333 time: 0.1709s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0025 loss_train: 0.6823 acc_train: 0.5641 loss_val: 0.6141 acc_val: 0.7333 time: 0.1866s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 

Epoch: 0049 loss_train: 0.6552 acc_train: 0.6667 loss_val: 0.6181 acc_val: 0.7000 time: 0.0746s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0050 loss_train: 0.6466 acc_train: 0.6538 loss_val: 0.6202 acc_val: 0.7667 time: 0.0911s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0051 loss_train: 0.6546 acc_train: 0.6667 loss_val: 0.6159 acc_val: 0.7333 time: 0.1495s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1

Epoch: 0072 loss_train: 0.6267 acc_train: 0.6795 loss_val: 0.5944 acc_val: 0.7333 time: 0.1081s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0073 loss_train: 0.6262 acc_train: 0.6795 loss_val: 0.5821 acc_val: 0.7333 time: 0.1514s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0074 loss_train: 0.6273 acc_train: 0.6923 loss_val: 0.5994 acc_val: 0.7333 time: 0.1960s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1

Epoch: 0095 loss_train: 0.6018 acc_train: 0.7051 loss_val: 0.5867 acc_val: 0.7333 time: 0.1115s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0096 loss_train: 0.6027 acc_train: 0.7051 loss_val: 0.5684 acc_val: 0.7333 time: 0.1154s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0097 loss_train: 0.6126 acc_train: 0.7564 loss_val: 0.5839 acc_val: 0.7667 time: 0.1123s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1

tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0119 loss_train: 0.6174 acc_train: 0.7436 loss_val: 0.5878 acc_val: 0.7333 time: 0.1127s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0120 loss_train: 0.6005 acc_train: 0.7179 loss_val: 0.5817 acc_val: 0.8000 time: 0.0907s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 

Epoch: 0142 loss_train: 0.6038 acc_train: 0.7308 loss_val: 0.5700 acc_val: 0.7667 time: 0.1001s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0143 loss_train: 0.6054 acc_train: 0.7308 loss_val: 0.5825 acc_val: 0.7333 time: 0.0813s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0144 loss_train: 0.5952 acc_train: 0.7564 loss_val: 0.5817 acc_val: 0.8000 time: 0.0667s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1

tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0167 loss_train: 0.6011 acc_train: 0.7564 loss_val: 0.6087 acc_val: 0.7667 time: 0.0774s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0168 loss_train: 0.6019 acc_train: 0.7821 loss_val: 0.5924 acc_val: 0.7667 time: 0.0651s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 

Epoch: 0189 loss_train: 0.6012 acc_train: 0.7436 loss_val: 0.5893 acc_val: 0.7667 time: 0.0696s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0190 loss_train: 0.6022 acc_train: 0.7564 loss_val: 0.5708 acc_val: 0.7667 time: 0.0647s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 0])
Epoch: 0191 loss_train: 0.6056 acc_train: 0.7436 loss_val: 0.6015 acc_val: 0.7667 time: 0.0619s
tensor([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1